In [2]:
import os
import asyncio
from dotenv import load_dotenv
from telegram import Update, ForceReply
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, CallbackContext
from pymongo import MongoClient

load_dotenv()

TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
MONGODB_URI = os.getenv("MONGODB_URI")

if not all([TELEGRAM_BOT_TOKEN, MONGODB_URI]):  
    raise ValueError("Missing required environment variables (TELEGRAM_BOT_TOKEN, MONGODB_URI).")

client = MongoClient(MONGODB_URI)
db = client["telegram_bot"]
users_collection = db["users"]

async def start(update: Update, context: CallbackContext):
    user = update.message.from_user
    user_data = {
        "first_name": user.first_name,
        "username": user.username,
        "chat_id": user.id
    }

    if users_collection.find_one({"chat_id": user.id}):
        await update.message.reply_text("Welcome back!")  # User already registered
    else:
        users_collection.insert_one(user_data)
        await update.message.reply_text("Welcome! Please share your contact information to complete registration.",
                                        reply_markup=ForceReply(selective=True))  # Request contact


async def handle_contact(update: Update, context: CallbackContext):
    if update.message.contact:  # Check if contact information is provided
        user = update.message.from_user
        contact = update.message.contact
        phone_number = contact.phone_number

        users_collection.update_one(
            {"chat_id": user.id},
            {"$set": {"phone_number": phone_number}}
        )
        await update.message.reply_text("Thank you for registering! Your phone number has been saved.")
    else:
        await update.message.reply_text("Please share your contact information using the 'Contact' button.")


async def main() -> None:
    application = ApplicationBuilder().token(TELEGRAM_BOT_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.CONTACT, handle_contact))  # Handle contact information

    try:
        await application.initialize()
        await application.start_polling()
        await application.idle()
    finally:
        client.close()


if __name__ == '__main__':
    asyncio.run(main())

ValueError: Missing required environment variables (TELEGRAM_BOT_TOKEN, MONGODB_URI).

In [3]:
TELEGRAM_BOT_TOKEN = "YOUR_ACTUAL_TELEGRAM_BOT_TOKEN"  # **REPLACE THIS**
MONGODB_URI = "YOUR_ACTUAL_MONGODB_URI"            # **REPLACE THIS**

print(f"TELEGRAM_BOT_TOKEN: {TELEGRAM_BOT_TOKEN}")
print(f"MONGODB_URI: {MONGODB_URI}")

if not all([TELEGRAM_BOT_TOKEN, MONGODB_URI]):
    raise ValueError("TELEGRAM_BOT_TOKEN or MONGODB_URI are not set. Check the values set in this cell.")

print("Variables set successfully!")

TELEGRAM_BOT_TOKEN: YOUR_ACTUAL_TELEGRAM_BOT_TOKEN
MONGODB_URI: YOUR_ACTUAL_MONGODB_URI
Variables set successfully!


In [4]:
from pymongo import MongoClient

try:
    client = MongoClient(MONGODB_URI)  # Use the variable from the previous cell
    db = client["test_database"]  # Try connecting to a test database
    collection = db["test_collection"]  # Try accessing a test collection

    # Try a simple operation (inserting a document)
    test_document = {"test": "success"}
    collection.insert_one(test_document)
    print("MongoDB connection and test successful!")

    client.close()  # Close the connection

except Exception as e:
    print(f"MongoDB connection failed: {e}")

MongoDB connection failed: your_actual_mongodb_uri:27017: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 679b6b82e6e7078e7eccd589, topology_type: Unknown, servers: [<ServerDescription ('your_actual_mongodb_uri', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('your_actual_mongodb_uri:27017: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


In [5]:
from telegram import Bot

try:
    bot = Bot(token=TELEGRAM_BOT_TOKEN)  # Use the variable from the first cell
    print("Telegram connection successful!")

    # You can try a simple Telegram API call here if you want:
    # me = bot.get_me()
    # print(me)

except Exception as e:
    print(f"Telegram connection failed: {e}")
    

Telegram connection successful!


In [6]:
from pymongo import MongoClient

try:
    client = MongoClient("mongodb://localhost:27017/")  # Try connecting to localhost
    print("MongoDB connection successful!")
    client.close()
except Exception as e:
    print(f"MongoDB connection failed: {e}")

MongoDB connection successful!


In [12]:
import os
print(f"Current working directory: {os.getcwd()}")

Current working directory: C:\telegram
